In [9]:
import pandas as pd

In [10]:
df = pd.read_excel('ready_for_filtering.xlsx')
df.head()

,Carimbo de data/hora,id_integer,role,years_exp,org_size,agile_methods,use_metrics_planning,use_metrics_review,use_metrics_weekly,use_metrics_daily,use_metrics_retro,metrics_category,sanitized_metrics,combined
0,17:34:49,1,Product manager,0 a 5,Microempresa,"Scrum, Kanban",Sim,Não,Sim,Não,Não,"Cronograma e progresso, Produto, Tecnologia, C...",Ideal team capacity,"productmanager0a5microempresascrum,kanbansimnã..."
1,17:57:23,2,Desenvolvedor(a),0 a 5,Microempresa,"Scrum, Kanban",Sim,Sim,Não,Não,Não,"Cronograma e progresso, Produto, Cliente",NPS,"desenvolvedor(a)0a5microempresascrum,kanbansim..."
2,17:57:23,2,Desenvolvedor(a),0 a 5,Microempresa,"Scrum, Kanban",Sim,Sim,Não,Não,Não,"Cronograma e progresso, Produto, Cliente",Burndown,"desenvolvedor(a)0a5microempresascrum,kanbansim..."
3,17:57:23,2,Desenvolvedor(a),0 a 5,Microempresa,"Scrum, Kanban",Sim,Sim,Não,Não,Não,"Cronograma e progresso, Produto, Cliente",Outstanding bugs,"desenvolvedor(a)0a5microempresascrum,kanbansim..."
4,17:57:23,2,Desenvolvedor(a),0 a 5,Microempresa,"Scrum, Kanban",Sim,Sim,Não,Não,Não,"Cronograma e progresso, Produto, Cliente",Throughput,"desenvolvedor(a)0a5microempresascrum,kanbansim..."


In [13]:
df['sanitized_metrics'] = df['sanitized_metrics'].fillna('')


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [15]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['sanitized_metrics'])

In [16]:
# Verificar as primeiras features extraídas
print(vectorizer.get_feature_names_out()[:10])

['active' 'agregado' 'and' 'available' 'avg' 'bugs' 'burndown' 'cac'
 'capacity' 'cfd']


In [17]:
from sklearn.cluster import KMeans


In [18]:
# Definir o número de clusters (n_clusters pode ser ajustado conforme o seu dataset)
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
df['cluster'] = kmeans.fit_predict(X)

In [19]:
df[['sanitized_metrics', 'cluster']].head()

,sanitized_metrics,cluster
0,Ideal team capacity,1
1,NPS,1
2,Burndown,4
3,Outstanding bugs,1
4,Throughput,1


In [20]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Similaridade de cosseno entre todas as métricas
cosine_sim = cosine_similarity(X, X)

# Função para recomendar métricas
def recommend_metrics(index, sim_matrix, df, top_n=5):
    # Obter as métricas mais similares
    similar_indices = np.argsort(-sim_matrix[index])[:top_n]
    
    # Retornar as métricas recomendadas
    return df.iloc[similar_indices]['sanitized_metrics']

# Exemplo de recomendação para um gerente de projeto
recommendations = recommend_metrics(0, cosine_sim, df)
recommendations

0      Ideal team capacity
526    Ideal team capacity
491    Ideal team capacity
499    Ideal team capacity
380    Ideal team capacity
Name: sanitized_metrics, dtype: object

In [23]:
# Concatenar outras features
X_additional = pd.get_dummies(df[['role', 'years_exp', 'org_size']], drop_first=True)
X_combined = pd.concat([pd.DataFrame(X.toarray()), X_additional.reset_index(drop=True)], axis=1)

X_combined.columns = X_combined.columns.astype(str)

# Aplicar o clustering novamente com as novas features
df['cluster'] = kmeans.fit_predict(X_combined)
